# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()["messages"])["content"]

{'model': 'llama3.2', 'created_at': '2024-12-13T20:14:59.793521565Z', 'message': {'role': 'assistant', 'content': 'Generative AI has numerous business applications across various industries. Here are some examples:\n\n1. **Content Generation**: Generative AI can create high-quality content such as articles, blog posts, social media posts, and even entire books. This can be used for content marketing, social media management, or even automating content creation.\n2. **Product Design and Development**: Generative AI can be used to design products, such as furniture, architecture, and product prototypes. It can also aid in the development of new product features and functionalities.\n3. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequently asked questions, and routing complex queries to human representatives.\n4. **Marketing and Advertising**: Generative AI can create personalized ads, social media posts, and even entire mark

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources on content creation.
2. **Marketing Automation**: Generative AI can be used to personalize marketing messages, create targeted ads, and automate email campaigns.
3. **Customer Service Chatbots**: AI-powered chatbots can provide 24/7 customer support, respond to queries, and help with simple issues, freeing up human agents for more complex problems.
4. **Product Design**: Generative AI can be used to design new products, such as furniture, jewelry, or electronics, by generating designs based on user input and preferences.
5. **Supply Chain Optimization**: AI can analyze data from various sources to optimize supply chain operations, predict demand, and identify potential bottlenecks.
6. **Data Analysis**: Generative AI ca

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [7]:
class Website:
    def __init__(self, url):
        self.url = url
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for HTTP errors
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract title
            self.title = soup.title.string if soup.title else "No title found"
            
            # Clean up the body content
            if soup.body:
                for irrelevant in soup.body(['script', 'style', 'img', 'input']):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = "No body content found"
        except requests.exceptions.RequestException as e:
            self.title = "Error fetching page"
            self.text = str(e)


In [9]:
webpage = Website("https://www.cnn.com/world/live-news/syria-civil-war-12-08-2024/index.html")
print("This is the title of the website-->", webpage.title)
print("Text of the website starts here ", webpage.text)

This is the title of the website--> December 8, 2024: Syria civil war news | CNN
Text of the website starts here  CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
World
Africa
Americas
Asia
Australia
China
Europe
India
Middle East
United Kingdom
More
Africa
Americas
Asia
Australia
China
Europe
India
Middle East
United Kingdom
Watch
Listen
Live TV
Subscribe
Sign in
My Account
Settings
Topics You Follow
Sign Out
Your CNN account
Sign in to your

In [10]:
# Define the prompt 

system_prompt = "You are an assistant that analyzes the contents of a website \
    and provides a short summary , ignoring text that might be navigation related .\
        respond in markdown"

In [15]:
def user_prompt_for(website):
    # Handle missing title and text gracefully
    title = website.title if website.title else "No title available"
    content = website.text if website.text else "No content available"
    
    # Create the user prompt
    user_prompt = f"You are looking at a website titled: **{title}**\n\n"
    user_prompt += "The contents of this website are as follows. Please provide a short summary of this website in markdown. "
    user_prompt += "If it includes news or announcements, then summarize these too.\n\n"
    
    # Truncate content if too long
    '''max_length = 5000  # Adjust based on your needs
    if len(content) > max_length:
        content = content[:max_length] + "\n\n[Content truncated for brevity...]"
    '''
    user_prompt += content
    return user_prompt


In [16]:
print(user_prompt_for(webpage))

You are looking at a website titled: **December 8, 2024: Syria civil war news | CNN**

The contents of this website are as follows. Please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
World
Africa
Americas
Asia
Australia
China
Europe
India
Middle East
United Kingdom
More
Africa
Americas
Asia
Australia
China
Europe
India
Mi

In [17]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
messages_for(webpage)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website     and provides a short summary , ignoring text that might be navigation related .        respond in markdown'},
 {'role': 'user',
  'content': 'You are looking at a website titled: **December 8, 2024: Syria civil war news | CNN**\n\nThe contents of this website are as follows. Please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nCNN values your feedback\n1. How relevant is this ad to you?\n2. Did you encounter any technical issues?\nVideo player was slow to load content\nVideo content never loaded\nAd froze or did not finish loading\nVideo content did not start after ad\nAudio on ad was too loud\nOther issues\nAd never loaded\nAd prevented/slowed the page from loading\nContent moved around while ad loaded\nAd was repetitive to ads I\'ve seen previously\nOther issues\nCancel\nSubmit\nThank You!\nYour effort and contrib

In [23]:
def summarize(url):
    website = Website(url)
    response =  ollama.chat(
        model=MODEL,
        messages=messages_for(website))
    return response['message']['content']

In [24]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [25]:
display_summary("https://edwarddonner.com")

**Summary**
================

*   **Website Overview**: The website is about Edward Donner, a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
*   **LLM Arena**: A unique arena where LLMs (Large Language Models) compete against each other in a battle of diplomacy and deviousness, showcasing the capabilities of these models.
*   **News and Announcements**:
    *   Mastering AI and LLM Engineering - Resources
    *   From Software Engineer to AI Data Scientist - resources
    *   Outsmart LLM Arena - a battle of diplomacy and deviousness
    *   Choosing the Right LLM: Toolkit and Resources
    *   November 13, 2024 update available